In [1]:
import networkx as nx
import numpy as np
from scipy.optimize import linprog
from copy import deepcopy
# from math import factorial, exp

In [2]:
# %run "attack_graph_MARA.ipynb"

## Pre-Processing

Please note this difference:
Adding virtual start node: We ADD a new virtual starting node and connect it to existing root (starting nodes)
Adding virtual target node: we REMOVE the old target nodes and replace them with one joint virtual target node

### Add Virtual starting node

### Debug Methods

In [3]:
def find_and_add_entry_node(graph):
    # First identify the original root nodes
    original_roots = [n for n, deg in graph.in_degree() if deg == 0]
    
    if len(original_roots) > 1:
        # add virtual entry node
        entry = 0  # virtual entry node
        graph.add_node(entry)
        for r in original_roots:
            graph.add_edge(entry, r, weight=1)
        return entry, graph, original_roots
    else:
        # Only one root, use it as entry
        entry = original_roots[0]
        return entry, graph, original_roots

### Add Virtual Target Node 

In [4]:
# def contract_target_nodes_r_style(graph):    
#     # Identify targets
#     targets = [n for n, deg in graph.out_degree() if deg == 0]
#     if len(targets) <= 1:
#         return graph
    
#     # Build a label that matches R's style
#     merged_label = f"c({','.join(str(t) for t in targets)})"
    
#     main_target = targets[0]
#     nx.relabel_nodes(graph, {main_target: merged_label}, copy=False)
    
#     for t in targets[1:]:
#         # Merge node 't' into the new label node, replicating edges
#         if graph.has_node(t):
#             graph = nx.contracted_nodes(graph, merged_label, t, self_loops=False)
    
#     return graph

In [5]:
# def contract_target_nodes_r_style(graph):
#     """
#     Replicates the R approach: merges all target nodes (out_degree=0) 
#     into a single 'super-node' but keeps the label that merges them 
#     (like 'c(12,13,14,16)') so that routes still show multiple steps.

#     Returns the modified graph with a single merged target node that 
#     has the original label, e.g. 'c(12,13,14,16)'.
#     """
    
#     # Let's Identify all out-degree=0 (terminal) nodes
#     targets = [n for n, deg in graph.out_degree() if deg == 0]
#     if len(targets) <= 1:
#         # Nothing to merge if 0 or 1 final target
#         return graph
    
#     # Build a single label: e.g. "c(12,13,14,16)"
#     merged_label = f"c({','.join(str(t) for t in targets)})"
    
#     # Pick one 'main' target to rename
#     main_target = targets[0]
#     nx.relabel_nodes(graph, {main_target: merged_label}, copy=False)
    
#     # Now let's contract the others into this new label to mirror R style
#     for t in targets[1:]:
#         if graph.has_node(t):
#             # contract t into merged_label
#             graph = nx.contracted_nodes(graph, merged_label, t, self_loops=False)
    
#     return graph

In [6]:
# def preprocess_graph(graph):
#     """
#     Preprocesses the graph by:
#       1) Identifying original root nodes.
#       2) Adding a virtual start node if multiple roots exist.
#       3) Redirecting all final targets to a *single* new numeric node (not "contract").
#          This keeps intermediate nodes near the end of paths, consistent with older expansions.

#     Returns:
#       (entry_node, processed_graph, original_roots)
#     """
    
#     # Identify original root nodes
#     original_roots = [n for n, deg in graph.in_degree() if deg == 0]
    
#     # Add virtual start node if needed
#     if len(original_roots) > 1:
#         entry_node = 0
#         while entry_node in graph.nodes():
#             entry_node -= 1  # keep subtracting 1 until we find an unused integer
#         graph.add_node(entry_node)
        
#         # Connect it to each root with weight=1, prob=1.0 (or prob=None)
#         for r in original_roots:
#             graph.add_edge(entry_node, r, prob=1.0, weight=0.0)
#     elif len(original_roots) == 1:
#         entry_node = original_roots[0]
#     else:
#         # No roots at all
#         entry_node = None
    
#     targets = [n for n, deg in graph.out_degree() if deg == 0]
#     if len(targets) > 1:
#         new_target = max(graph.nodes()) + 1 if graph.nodes() else 1
#         # build a new DiGraph to hold the redirected edges
#         new_graph = nx.DiGraph()
        
#         # add all non-target nodes
#         non_targets = [n for n in graph.nodes() if n not in targets]
#         new_graph.add_nodes_from(non_targets)
        
#         edge_properties = {}
#         for u, v, data in graph.edges(data=True):
#             if v in targets:
#                 key = (u, new_target)
#             elif u not in targets:
#                 key = (u, v)
#             else:
#                 continue
            
#             if key not in edge_properties:
#                 edge_properties[key] = data.copy()
#             else:
#                 # Logic to keep the edge with higher 'prob' value
#                 old_prob = edge_properties[key].get('prob', 0)
#                 new_prob = data.get('prob', 0)
#                 if new_prob > old_prob:
#                     edge_properties[key] = data.copy()
        
#         # now add edges to new_graph
#         for (u, v), d in edge_properties.items():
#             new_graph.add_edge(u, v, **d)
        
#         # finally add the new target node
#         new_graph.add_node(new_target)
        
#         # this new_graph is our final
#         graph = new_graph
    
#     # If we have exactly 1 or 0 targets, do nothing 
#     return entry_node, graph, original_roots


In [17]:
def merge_targets_with_multi_edges(orig_graph):
    targets = [n for n, deg in orig_graph.out_degree() if deg == 0]
    if len(targets) <= 1:
        return orig_graph

    merged_label = "c(" + ",".join(str(t) for t in targets) + ")"
    newG = nx.MultiDiGraph()
    
    # Add non-target nodes
    non_targets = [n for n in orig_graph.nodes() if n not in targets]
    newG.add_nodes_from(non_targets)
    newG.add_node(merged_label)
    
    # Track edges from each predecessor to each target
    pred_target_edges = {}
    
    # Collect edges to targets
    for u, v, data in orig_graph.edges(data=True):
        if v in targets:
            if u not in pred_target_edges:
                pred_target_edges[u] = {}
            weight = data.get('weight', 1)
            if weight not in pred_target_edges[u]:
                pred_target_edges[u][weight] = set()
            pred_target_edges[u][weight].add(v)
    
    # Create edges based on pattern
    for u, weight_targets in pred_target_edges.items():
        if len(weight_targets) == 1:  # Only one unique weight
            weight = list(weight_targets.keys())[0]
            # If this weight connected to multiple original targets
            num_targets = len(list(weight_targets.values())[0])
            if num_targets > 1:
                # Create one edge per original target
                for _ in range(num_targets):
                    newG.add_edge(u, merged_label, weight=weight)
            else:
                # Single target, single edge
                newG.add_edge(u, merged_label, weight=weight)
        else:
            # Multiple weights - create one edge per weight
            for weight in sorted(weight_targets.keys()):
                newG.add_edge(u, merged_label, weight=weight)
    
    # Add non-target edges
    for u, v, data in orig_graph.edges(data=True):
        if v not in targets and u not in targets:
            newG.add_edge(u, v, **data)
    
    return newG

## Static elements preparation for the Game

### Generate game elements

In [8]:
def generate_game_elements(graph, entry_node, original_roots):
    """
    Generate game elements after we have the final graph (with single entry and single target).
    Modified to handle parallel edges correctly by consolidating paths that only differ by parallel edges.
    """
    # determine the target node
    target_list = [n for n,d in graph.out_degree() if d == 0]
    if len(target_list) != 1:
        print("WARNING: Expected exactly one target node after contraction. Found:", target_list)
    
    # Find all routes using nx.all_simple_paths but consolidate parallel paths
    raw_routes = list(nx.all_simple_paths(graph, entry_node, target_list[0]))
    
    # Consolidate paths that are identical except for parallel edges
    consolidated_routes = []
    seen_paths = set()
    
    for path in raw_routes:
        # Convert path to a tuple of nodes (ignoring parallel edges)
        path_key = tuple(path)
        if path_key not in seen_paths:
            seen_paths.add(path_key)
            consolidated_routes.append(list(path))
    
    routes = consolidated_routes

    # Get V as unique nodes from all routes
    V = sorted(set(node for path in routes for node in path), key=str)

    # Get topological ordering
    topo_all = list(nx.topological_sort(graph))
    node_order = [n for n in topo_all if n in V]
    
    # Create excluded nodes set for as1:
    # a) virtual start node (entry_node)
    # b) virtual target node (target_list)
    # c) original root nodes (original_roots)
    excluded = {entry_node} | set(target_list) | set(original_roots)
    
    # Generate as1 by excluding all necessary nodes
    as1 = [n for n in V if n not in excluded]
    
    # as2 are the consolidated routes
    as2 = routes

    # For adv_list, we only exclude entry and target nodes (NOT original roots)
    adv_list = [n for n in V if n not in (set([entry_node]) | set(target_list))]
    
    # Calculate theta
    theta = {loc: 1/len(adv_list) for loc in adv_list} if adv_list else {}
    
    return routes, V, as1, as2, target_list, node_order

In [9]:
# Creates a list of all possible attacker locations (excluding entry and target nodes)
# Assigns equal probability (1/n) to each possible location in a dictionary
# Counts the total number of attack paths

# Returns:
# adv_list: List of nodes where attacker could be
# theta: Dictionary mapping each possible location to its probability (all equal)
# m: Number of attack paths

def setup_game_parameters(V, routes, entry_node, target_list):
    # prepare all the possible locations of attacker (avatars)
    adv_list = [n for n in V if n not in [entry_node] + target_list]
    if len(adv_list) == 0:
        print("WARNING: No adversary intermediate locations found. Check graph structure.")
    
    # Create a dictionary that assigns equal probability to each possible attacker location
    theta = {loc: 1/len(adv_list) for loc in adv_list}
    # Number of attack paths
    m = len(routes)
    return adv_list, theta, m

## Dynamic elements preparation for the Game

### Method to Calculate the Pay_Offs 

#### Some Explanations

In [10]:
# This method returns "payoffs":
# payoffs = [
#     {'dpdf': [0.1, 0.2, 0.7], 'support': [1,2,3], ...},  # for check1 + path1
#     {'dpdf': [0.3, 0.5, 0.2], 'support': [1,2,3], ...},  # for check1 + path2
#     {'dpdf': [0.4, 0.4, 0.2], 'support': [1,2,3], ...},  # for check2 + path1
#     {'dpdf': [0.1, 0.1, 0.8], 'support': [1,2,3], ...}   # for check2 + path2
# ]

Let's look at:
{'dpdf': [0.1, 0.2, 0.7], 'support': [1,2,3]}  # for check1 + path1

This means:

When defender checks at location 1, and attacker uses path1:

10% chance attacker is at first node (0.1)

20% chance attacker is at second node (0.2)

70% chance attacker reached the target node! (0.7)

Note: Support [1,2,3] are NOT the node number. I think of them as "steps away from start" rather than actual node IDs.

That's why we take dpdf[-1] for our payoff matrix - it's the probability the attacker successfully reaches the target node under this check+path combination.

##### Method necessary to calculate Pay offs within

This method is mostly useless and can be removed long term. In my current implementation all we care about is the last entry of U = [0.17, 0.10, 0.63, 0.10, 1e-7] which represents the prob. of the attacker to reach the target node.

And per one U (Node Check & Attack Path pair), we extract only this one value for the final pay off matrix. 
lossDistribution() does not add anything of value.

In [11]:
# def lossDistribution(U):
#     U = U / np.sum(U)
#     support = np.arange(1, len(U)+1)
#     dpdf = U
#     cdf = np.cumsum(U)
#     tail = 1 - cdf + U
#     return {
#         'support': support,
#         'dpdf': dpdf,
#         'cdf': cdf,
#         'tail': tail
#     }

#### End of Explanations

In [12]:
def calculate_payoff_distribution(graph, as1, as2, V, adv_list, theta, random_steps_fn, 
                                  attack_rate, defense_rate, node_order):
    payoffs = []

    for check in as1:
        for path in as2:
            U = np.zeros(len(V))

            print(f"\n++++++++++++++++++++++++++++++++")
            print(f"attack_rate = {attack_rate}, defense_rate = {defense_rate}")
            print(f"--- Starting payoff calc for check = {check}, path = {path} ---\n")

            for avatar in adv_list:
                L = np.zeros(len(V))

                if avatar in path:
                    start_idx = path.index(avatar)
                    route = path[start_idx:]
                    print(f"\nProcessing avatar {avatar}:")
                    print(f"Route from avatar: {route}")
                    
                    pdf_d = random_steps_fn(route, attack_rate, defense_rate, graph)
                    print(f"PDF for entire route: {pdf_d}")

                    if check in route:
                        check_idx = route.index(check)
                        cutPoint = min(check_idx + 1, len(route))
                    else:
                        cutPoint = len(route)
                    print(f"Cut point: {cutPoint}")

                    pdf_subset = pdf_d[:cutPoint]
                    if np.sum(pdf_subset) < 1e-15:
                        payoffDistr = np.zeros(cutPoint)
                        payoffDistr[-1] = 1.0
                    else:
                        payoffDistr = pdf_subset / np.sum(pdf_subset)
                    
                    print(f"PDF subset: {pdf_subset}")
                    print(f"Payoff distribution: {payoffDistr}")

                    # Fill L for each node in route_subset
                    route_subset = route[:cutPoint]
                    print(f"Route subset: {route_subset}")
                    # for idx_node, node in enumerate(route_subset):
                    #     L[V.index(node)] = pdf_d[idx_node]

                    for idx_node, node in enumerate(route_subset):
                        L[V.index(node)] = payoffDistr[idx_node]
                    
                    print("L distribution for this avatar (BEFORE weighting by Theta):")
                    for idx_l, val in enumerate(L):
                        if val > 1e-10:
                            print(f"  Node {V[idx_l]} : {val}")
                else:
                    L[V.index(avatar)] = 1.0
                    print(f"\nProcessing avatar {avatar} (not in path):")
                    print(f"L[{avatar}] = 1.0")

                print(f"\nTheta[{avatar}] = {theta[avatar]}")
                U += theta[avatar] * L
                print("Current U after adding this avatar's contribution:")
                for idx_u, val in enumerate(U):
                    if val > 1e-10:
                        print(f"  Node {V[idx_u]} : {val}")

            print(f"\n--- Aggregated U for check={check}, path={path} (BEFORE normalization) ---")
            for idx_u, val in enumerate(U):
                if val > 1e-10:
                    print(f"  Node {V[idx_u]} : {val}")

            U_sum = np.sum(U)
            if U_sum < 1e-15:
                U = np.full_like(U, 1e-7)
            else:
                U /= U_sum
                U = np.where(U < 1e-7, 1e-7, U)
            
            node_positions = [V.index(n) for n in node_order]
            U = U[node_positions]

            print(f"\n--- Normalized U for check={check}, path={path} ---")
            for idx_u2, val2 in enumerate(U):
                if val2 > 1e-10:
                    print(f"  Node {node_order[idx_u2]} : {val2}")

            ld = {
                'dpdf': U,
                'support': range(1, len(U) + 1),
                'cdf': np.cumsum(U),
                'tail': 1 - np.cumsum(U) + U
            }
            payoffs.append(ld)

    return payoffs

### Method for finding optimal solutions

#### Some Explanations

This method solves a zero-sum game using linear programming. It first creates a proper payoff matrix from the loss distributions, then solves two LPs: one for defender minimizing attacker success probability, and one for attacker maximizing it. Both optimizations should yield the same equilibrium value.

- Constructs payoff matrix
- Solves LP for defender FIRST (minimizing attacker success)
- Then solves LP for attacker (maximizing their success)
- Verifies both values match (equilibrium property)

The method returns a dictionary: 

{

    'optimal_defense': {3: 0.6, 4: 0.4},  # defend node 3 with 60% probability, node 4 with 40%                     
    'attacker_strategy': [0.3, 0.5, 0.2], # use path 1 with 30% probability, path 2 with 50%, etc.
    'defender_success': 0.128,           # defender can keep attacker success ≤ 0.128              
    'attacker_success': 0.128            # attacker can achieve at least 0.128
                                            
}

#### End of Explanations

In [13]:
def solve_game(payoffs, as1, as2):
    n = len(as1)
    m = len(as2)
    
    # Create payoff matrix
    payoff_matrix = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            idx = i*m + j
            ld = payoffs[idx]
            payoff_matrix[i, j] = ld['dpdf'][-1]
    
    ### Start Defender's optimization ###
    c = np.zeros(n+1)
    c[0] = 1.0
    
    A_ub = np.zeros((m, n+1))
    b_ub = np.zeros(m)
    for j in range(m):
        A_ub[j,0] = -1.0
        for i in range(n):
            A_ub[j,i+1] = payoff_matrix[i,j]
            
    A_eq = np.zeros((1, n+1))
    A_eq[0,1:] = 1.0
    b_eq = np.array([1.0])
    
    bounds = [(0,None)]*(n+1)
    
    v_defender = None
    v_attacker = None
    
    # Solve the LP
    res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
    
    ### End Defender's optimization ###
    if res.success:
        # Extract the results for later logging
        v_defender = res.x[0]
        x_def = res.x[1:]
        
        ### Start Attacker's optimization ###
        c_att = np.zeros(m+1)
        c_att[0] = -1.0
        
        A_ub_att = np.zeros((n, m+1))
        b_ub_att = np.zeros(n)
        for i in range(n):
            A_ub_att[i,0] = 1.0
            for j in range(m):
                A_ub_att[i,j+1] = -payoff_matrix[i,j]
                
        A_eq_att = np.zeros((1, m+1))
        A_eq_att[0,1:] = 1.0
        b_eq_att = np.array([1.0])
        
        bounds_att = [(0,None)]*(m+1)
        res_att = linprog(c_att, A_ub=A_ub_att, b_ub=b_ub_att, 
                         A_eq=A_eq_att, b_eq=b_eq_att, bounds=bounds_att)
        
        ### End Attacker's optimization ###
        
        if res_att.success:
            # Extract attacker results for later logging
            y_att = res_att.x[1:]
            v_attacker = res_att.x[0]   # new - remove the negative sign because c_att[0] = -1.0
            
            # Now both values are defined, we can check
            if abs(v_defender - v_attacker) > 1e-5:
                logger.info("\nWarning: Defender and attacker values don't match!")
                logger.info(f"Defender value: {v_defender:.6f}")
                logger.info(f"Attacker value: {v_attacker:.6f}")
            
            return {
                'optimal_defense': dict(zip(as1, x_def)),
                'attacker_strategy': y_att,
                'defender_success': v_defender,
                'attacker_success': v_attacker
            }
    
    logger.info("LP optimization failed")
    return None

### Method to run the Actual Game

#### Some Explanations

Key Steps:

1. Finds entry node in graph
2. Generates game elements (routes, nodes, defense locations) using generate_game_elements()
3. Sets up game parameters (adversary locations, probabilities) using setup_game_parameters()

Then For each attack/defense rate combination:

5. Calculates payoff distributions
6. Solves game using solve_game()
7. Logs results (optimal strategies, success probabilities)

Returns:
Nothing - this method only logs results to the logger
All output goes to log file 

#### End of Explanations

In [14]:
def print_debug_info(graph, stage=""):
    print(f"\n{stage}:")
    print("Nodes:", list(graph.nodes()))
    print("Edges with weights:")
    if isinstance(graph, nx.MultiDiGraph):
        # For MultiDiGraph, we need to handle multiple edges between same nodes
        for u, v, key, data in graph.edges(data=True, keys=True):
            weight = data.get('weight', 1)  # default to 1 if no weight
            print(f"{u} -> {v} (key={key}) : {weight}")
    else:
        # Original printing for regular DiGraph
        for u, v, data in graph.edges(data=True):
            weight = data.get('weight', 1)  # default to 1 if no weight
            print(f"{u} -> {v} : {weight}")

In [15]:
def run_game(graph, attack_rate_list, defense_rate_list, random_steps_fn):
    virtual_entry_node, graph, original_roots = find_and_add_entry_node(graph)
    print_debug_info(graph, "Before merging targets")
    graph = merge_targets_with_multi_edges(graph)  # Only use this function
    print_debug_info(graph, "After merging targets")

    # virtual_entry_node, graph, original_roots = find_and_add_entry_node(G_multi)
    # print_debug_info(graph, "After merging targets")

    routes, V, as1, as2, target_list, node_order = generate_game_elements(graph, virtual_entry_node, original_roots)
    adv_list, theta, m = setup_game_parameters(V, routes, virtual_entry_node, target_list)
    
    if not defense_rate_list:
        defense_rate_list = [0]
    if not attack_rate_list:
        attack_rate_list = [0]
    
    for defenseRate in defense_rate_list:
        for attackRate in attack_rate_list:
            logger.info("\n++++++++++++++++++++++++++++++++")
            logger.info(f"\nThe virtual target nodeID is {target_list[0]}\n")
            logger.info(f"attack rate =  {attackRate} , defense rate =  {defenseRate} \n")
            logger.info("\tequilibrium for multiobjective security game (MOSG)\n")
            

            # changing this was important because the function is now called with the graph
            payoffs = calculate_payoff_distribution(
                graph, as1, as2, V, adv_list, theta, 
                random_steps_fn,  # Just pass the function directly
                attackRate, defenseRate, node_order
            )

            # debug_payoff_matrix(payoffs, as1, as2)
            
            eq = solve_game(payoffs, as1, as2)
            if eq is not None:
                logger.info("optimal defense strategy:")
                logger.info("         prob.")
                for node, prob in sorted(eq['optimal_defense'].items(), key=lambda x: str(x[0])):
                    logger.info(f"{node} {prob:.6e}")
                
                logger.info("\nworst case attack strategies per goal:")
                logger.info("          1")
                if 'attacker_strategy' in eq:
                    for idx, prob in enumerate(eq['attacker_strategy'], 1):
                        logger.info(f"{idx} {prob:.7f}")
                logger.info(f"[1] {eq['attacker_success']:.3f}")
                
                logger.info(f"\nDefender can keep attacker success below: {eq['defender_success']:.3f}")
                logger.info(f"Attacker can guarantee success probability of: {eq['attacker_success']:.3f}")

### Method to be called in different Notebook

In [16]:
# This method is supposed to be run in experiment_X.ipynb files. It is not a standalone script.
def main():
    # Prepare the graph
    work_graph = deepcopy(attack_graph)
    # entry_node, work_graph, original_roots = find_and_add_entry_node(work_graph)

    # work_graph = process_target_nodes(work_graph)

    # Run the game
    run_game(work_graph, attack_rate_list=attack_rate_list, defense_rate_list=defense_rate_list, random_steps_fn=random_steps)